# **Aula 1 - Preparando os Dados**

## **1.1 Apresentação**

## **1.2 Preparando o Ambiente**

### PySpark

PySpark é uma interface para Apache Spark em Python. Ele não apenas permite que você escreva aplicativos Spark usando APIs Python, mas também fornece o *shell* PySpark para analisar interativamente seus dados em um ambiente distribuído. O PySpark oferece suporte à maioria dos recursos do Spark, como Spark SQL, DataFrame, Streaming, MLlib (Machine Learning) e Spark Core.

<center><img src="https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/img-001.png"/></center>

#### **Spark SQL e DataFrame**

Spark SQL é um módulo Spark para processamento de dados estruturados. Ele fornece uma abstração de programação chamada DataFrame e também pode atuar como mecanismo de consulta SQL distribuído.

#### **Spark Streaming**

Executando em cima do Spark, o recurso de *streaming* no Apache Spark possibilita o uso de poderosas aplicações interativas e analíticas em *streaming* e dados históricos, enquanto herda a facilidade de uso do Spark e as características de tolerância a falhas.

#### **Spark MLlib**

Construído sobre o Spark, MLlib é uma biblioteca de aprendizado de máquina escalonável que fornece um conjunto uniforme de APIs de alto nível que ajudam os usuários a criar e ajustar *pipelines* de aprendizado de máquina práticos.

#### **Spark Core**

Spark Core é o mecanismo de execução geral subjacente para a plataforma Spark sobre o qual todas as outras funcionalidades são construídas. Ele fornece um RDD (*Resilient Distributed Dataset*) e recursos de computação na memória.

<font size=2>**Fonte:** [PySpark](https://spark.apache.org/docs/latest/api/python/index.html)</font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### SparkSession

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

<font size=2>**Fonte:** [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)</font>

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master('local[*]').appName("Classificação com Spark").getOrCreate()
spark

## **1.3 Carregamento dos Dados**

In [5]:
dados = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/4. Alura - Formacao Data Scientist/Curso 3 - Apache Spark classificacao/aula02/dados_clientes.csv',
                       sep=',',
                       header=True,
                       inferSchema=True)

In [6]:
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

In [7]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [8]:
dados.count()

10348

In [9]:
(dados
 .select('Churn')
 .groupBy('Churn')
 .count()
).show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [10]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



## **1.4 Transformando os Dados**

<font size=2>**Fonte:** [Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)</font>

In [11]:
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [12]:
from pyspark.sql import functions as f

In [13]:
# monta a tranformação para alterar os valores nas colunas binárias
todasColunas = [f.when(f.col(c)=='Sim', 1).otherwise(0).alias(c) for c in colunasBinarias]

# insere as demais colunas na lista "todasColunas"
for coluna in reversed(dados.columns):
  if coluna not in colunasBinarias:
    todasColunas.insert(0, coluna)
todasColunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [14]:
# aplica as transformações nas colunas binárias
dataset = dados.select(todasColunas)

In [15]:
# viosualiza o resultado da transformação
dataset.show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

## **1.5 Criando *Dummies***

In [16]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



In [17]:
dataset.select('Internet', 'TipoContrato', 'MetodoPagamento').show(5)

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
+-----------+------------+----------------+
only showing top 5 rows



In [18]:
# variaveis dummy da coluna Internet
(dataset
 .groupBy('id')
 .pivot('Internet')
 .agg(f.lit(1))
 .fillna(0)
).show(5)

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
+----+---+-----------+---+
only showing top 5 rows



In [19]:
internet = (dataset
            .groupBy('id')
            .pivot('Internet')
            .agg(f.lit(1))
            .fillna(0)
)

In [20]:
# variaveis dummy da coluna TipoContrato
(dataset
 .groupBy('id')
 .pivot('TipoContrato')
 .agg(f.lit(1))
 .fillna(0)    
).show(5)

+----+--------+-----------+-----+
|  id|DoisAnos|Mensalmente|UmAno|
+----+--------+-----------+-----+
|7993|       0|          1|    0|
|8592|       0|          1|    0|
|4519|       0|          0|    1|
|1088|       0|          1|    0|
|1238|       0|          1|    0|
+----+--------+-----------+-----+
only showing top 5 rows



In [21]:
tipo_contrato = (dataset
                .groupBy('id')
                .pivot('TipoContrato')
                .agg(f.lit(1))
                .fillna(0)    
)

In [22]:
# variaveis dummy da coluna MetodoPagamento
(dataset
 .groupBy('id')
 .pivot('MetodoPagamento')
 .agg(f.lit(1))
 .fillna(0)    
).show(5)

+----+------+----------------+-------------+-------------+
|  id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+----+------+----------------+-------------+-------------+
|3997|     0|               0|            1|            0|
|7554|     0|               1|            0|            0|
|6336|     0|               1|            0|            0|
|6357|     0|               1|            0|            0|
|9427|     0|               0|            1|            0|
+----+------+----------------+-------------+-------------+
only showing top 5 rows



In [23]:
metodo_pagamento = (dataset
                .groupBy('id')
                .pivot('MetodoPagamento')
                .agg(f.lit(1))
                .fillna(0)    
)

In [24]:
# faz o "merge" das colunas dummies com o dataset original
(dataset
 .join(internet, on='id', how='inner')
 .join(tipo_contrato, on='id', how='inner')
 .join(metodo_pagamento, on='id', how='inner')
).show(5)

+----+----------+---------------+-----------+------------+----------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+---+-----------+---+--------+-----------+-----+------+----------------+-------------+-------------+
|  id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|DSL|FibraOptica|Nao|DoisAnos|Mensalmente|UmAno|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+----+----------+---------------+-----------+------------+----------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+---+-----------+---+--------+---

In [25]:
dataset = (dataset
          .join(internet, on='id', how='inner')
          .join(tipo_contrato, on='id', how='inner')
          .join(metodo_pagamento, on='id', how='inner')
)

In [26]:
# renomeia as colunas criadas
dataset = (dataset
          .select('*',
                  f.col('DSL').alias('Internet_DSL'), 
                  f.col('FibraOptica').alias('Internet_FibraOptica'), 
                  f.col('Nao').alias('Internet_Nao'), 
                  f.col('Mensalmente').alias('TipoContrato_Mensalmente'), 
                  f.col('UmAno').alias('TipoContrato_UmAno'), 
                  f.col('DoisAnos').alias('TipoContrato_DoisAnos'), 
                  f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'), 
                  f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'), 
                  f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'), 
                  f.col('Boleto').alias('MetodoPagamento_Boleto')
                 )
)

In [27]:
# drop colunas
dataset = (dataset
           .drop('Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
                 'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
                 'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
           )
)

In [28]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

In [29]:
dataset.show(5)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

## **2.2 Preparação dos Dados**

In [30]:
dataset.show(5)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [31]:
from pyspark.ml.feature import VectorAssembler

In [32]:
dataset = dataset.withColumnRenamed('Churn', 'label')

In [33]:
# retira nome das colunas "label e "id" da lista de features
X = dataset.drop('label','id').columns
X[:5]

['Mais65anos', 'MesesDeContrato', 'MesesCobrados', 'Conjuge', 'Dependentes']

In [34]:
assembler = VectorAssembler(inputCols = X, outputCol='features')

In [35]:
dataset_prep = assembler.transform(dataset).select('label','features')

In [36]:
dataset_prep.show(5, truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------+
|label|features                                                                                                   |
+-----+-----------------------------------------------------------------------------------------------------------+
|1    |(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |
|1    |(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|0    |(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |
|0    |(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |
|1    |(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |
+-----+-----------------------------------------------------------------

## **2.3 Ajuste e Previsão - LogisticRegression()**

In [37]:
seed= 101

In [38]:
treino, teste = dataset_prep.randomSplit([0.7,0.3], seed=seed)

In [39]:
treino.count()

7206

In [40]:
teste.count()

3142

In [41]:
from pyspark.ml.classification import LogisticRegression

In [42]:
lr = LogisticRegression()

In [43]:
modelo_lr = lr.fit(treino)

In [44]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [45]:
previsoes_lr_teste.show(5, truncate=False)

+-----+----------------------------------------------------------------------------------------------------+------------------------------------------+----------------------------------------+----------+
|label|features                                                                                            |rawPrediction                             |probability                             |prediction|
+-----+----------------------------------------------------------------------------------------------------+------------------------------------------+----------------------------------------+----------+
|0    |(24,[0,1,2,3,4,5,6,7,9,10,11,14,18,22],[1.0,55.0,76.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|[3.112228143436698,-3.112228143436698]    |[0.9573943353759904,0.04260566462400961]|0.0       |
|0    |(24,[0,1,2,3,4,5,6,7,13,15,17,22],[1.0,24.0,79.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])             |[-0.11437769844422374,0.11437769844422374]|[0.47143670796424897,0.5285632920

## **2.4 Métricas - LogisticRegression()**

In [46]:
resumo_lr_treino = modelo_lr.summary

In [47]:
print("Acurácia: %f" % resumo_lr_treino.accuracy)
print("Precisão: %f" % resumo_lr_treino.precisionByLabel[1])
print("Recall: %f" % resumo_lr_treino.recallByLabel[1])
print("F1 score: %f" % resumo_lr_treino.fMeasureByLabel()[1])
print("ROC_AUC: %f" % resumo_lr_treino.areaUnderROC)

Acurácia: 0.787954
Precisão: 0.773404
Recall: 0.811384
F1 score: 0.791939
ROC_AUC: 0.870730


In [48]:
# Matriz de confusão
print('Matriz de Confusão:', '\n')

(previsoes_lr_teste
 .select('label','prediction')
 .groupBy('label','prediction')
 .count()
).show()

Matriz de Confusão: 

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  304|
|    0|       0.0| 1125|
|    1|       1.0| 1286|
|    0|       1.0|  427|
+-----+----------+-----+



## **2.5 Ajuste e Previsão - DecisionTreeClassifier()**

In [49]:
from pyspark.ml.classification import DecisionTreeClassifier

In [50]:
dtc = DecisionTreeClassifier(maxDepth = 5, maxBins = 32, seed=seed)

In [51]:
modelo_dtc = dtc.fit(treino)

In [52]:
previsoes_dtc_treino = modelo_dtc.transform(treino)

In [53]:
previsoes_dtc_treino.show(5)

+-----+--------------------+--------------+--------------------+----------+
|label|            features| rawPrediction|         probability|prediction|
+-----+--------------------+--------------+--------------------+----------+
|    0|(24,[0,1,2,3,4,5,...|[2084.0,327.0]|[0.86437163002903...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|[2084.0,327.0]|[0.86437163002903...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|    [24.0,2.0]|[0.92307692307692...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|    [24.0,2.0]|[0.92307692307692...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|    [24.0,2.0]|[0.92307692307692...|       0.0|
+-----+--------------------+--------------+--------------------+----------+
only showing top 5 rows



## **2.6 Métricas - DecisionTreeClassifier()**

In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [55]:
evaluator = MulticlassClassificationEvaluator()

In [56]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
# print("ROC_AUC: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "areaUnderROC"}))

Acurácia: 0.788926
Precisão: 0.797691
Recall: 0.771205
F1: 0.784225


In [57]:
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [58]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.772120
Precisão: 0.779055
Recall: 0.767296
F1: 0.773131


In [59]:
# Matriz de confusão
print('Matriz de Confusão:', '\n')

(previsoes_dtc_teste
 .select('label','prediction')
 .groupBy('label','prediction')
 .count()
).show()

Matriz de Confusão: 

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  370|
|    0|       0.0| 1206|
|    1|       1.0| 1220|
|    0|       1.0|  346|
+-----+----------+-----+



## **2.7 Ajuste e previsão - RandomForestClassifier()**

In [60]:
from pyspark.ml.classification import RandomForestClassifier

In [61]:
rfc = RandomForestClassifier(seed=seed)

In [62]:
modelo_rfc = rfc.fit(treino)

In [63]:
previsoes_rfc_treino = modelo_rfc.transform(treino)

In [64]:
previsoes_rfc_treino.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(24,[0,1,2,3,4,5,...|[14.5395757907657...|[0.72697878953828...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|[15.5752440358381...|[0.77876220179190...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|[7.94237350267947...|[0.39711867513397...|       1.0|
|    0|(24,[0,1,2,3,4,5,...|[7.96850326290079...|[0.39842516314503...|       1.0|
|    0|(24,[0,1,2,3,4,5,...|[8.18918381845635...|[0.40945919092281...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [65]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [66]:
previsoes_rfc_teste.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(24,[0,1,2,3,4,5,...|[15.9877734196032...|[0.79938867098016...|       0.0|
|    0|(24,[0,1,2,3,4,5,...|[7.93132996367620...|[0.39656649818381...|       1.0|
|    0|(24,[0,1,2,3,4,5,...|[8.52895101201303...|[0.42644755060065...|       1.0|
|    0|(24,[0,1,2,3,4,5,...|[8.94162007370314...|[0.44708100368515...|       1.0|
|    0|(24,[0,1,2,3,4,5,...|[8.32086691252480...|[0.41604334562624...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## **2.6 Métricas - RandomForestClassifier()**

In [67]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.769574
Precisão: 0.756213
Recall: 0.803774
F1: 0.779268


In [68]:
#Matriz de confusão
print('Matriz de Confusão:', '\n')

(previsoes_rfc_teste
 .select('label','prediction')
 .groupBy('label','prediction')
 .count()
).show()

Matriz de Confusão: 

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  312|
|    0|       0.0| 1140|
|    1|       1.0| 1278|
|    0|       1.0|  412|
+-----+----------+-----+



# **3. Cross Validation (CV)**

## **3.1 DecisionTreeClassifier - Cross Validation (CV)**

In [69]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [70]:
dtc = DecisionTreeClassifier(seed=seed)

In [71]:
# grid de parametros para otimização
grid = ParamGridBuilder() \
          .addGrid(dtc.maxDepth, [2,5,10]) \
          .addGrid(dtc.maxBins, [10,32,45]) \
          .build()

In [72]:
# avaliador
evaluator = MulticlassClassificationEvaluator()

In [73]:
# Validação cruzada
dtc_cv = CrossValidator(estimator = dtc,
                        estimatorParamMaps = grid,
                        evaluator = evaluator,
                        numFolds = 3,
                        seed=seed)

In [74]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [75]:
previsoes_dtc_cv = modelo_dtc_cv.transform(teste)

In [76]:
print("Decision Tree Classifier - Metrics: ")
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier - Metrics: 
Acurácia: 0.786760
Precisão: 0.760181
Recall: 0.845283
F1: 0.800476


In [77]:
#Matriz de confusão
print('Matriz de Confusão:', '\n')

(previsoes_dtc_cv
 .select('label','prediction')
 .groupBy('label','prediction')
 .count()
).show()

Matriz de Confusão: 

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  246|
|    0|       0.0| 1128|
|    1|       1.0| 1344|
|    0|       1.0|  424|
+-----+----------+-----+



## **3.2 RandomForestClassifier - Cross Validation (CV)**

In [78]:
rfc = RandomForestClassifier(seed=seed)

In [79]:
# grid de parametros para otimização
grid = ParamGridBuilder() \
          .addGrid(rfc.maxDepth, [10,12,14]) \
          .addGrid(rfc.maxBins, [45,55,65]) \
          .addGrid(rfc.numTrees, [50,65,80]) \
          .build()

In [80]:
rfc_cv = CrossValidator(estimator = rfc,
                        estimatorParamMaps = grid,
                        evaluator = evaluator,
                        numFolds = 3,
                        seed=seed)

In [81]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [82]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [83]:
print("Random Forest Classifier - Metrics: ")
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier - Metrics: 
Acurácia: 0.819542
Precisão: 0.798948
Recall: 0.859748
F1: 0.828234


In [84]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [85]:
print("melhor RandomForest maxDepth: ", melhor_modelo_rfc_cv.getMaxDepth())
print("melhor RandomForest maxBins: ", melhor_modelo_rfc_cv.getMaxBins())
print("melhor RandomForest numTrees: ", melhor_modelo_rfc_cv.getNumTrees)

melhor RandomForest maxDepth:  14
melhor RandomForest maxBins:  65
melhor RandomForest numTrees:  80


In [86]:
rfc_tuning = RandomForestClassifier(maxDepth=14, maxBins=65, numTrees=80, seed=seed)

In [87]:
modelo_rfc_tuning = rfc_tuning.fit(dataset_prep)

In [88]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

In [89]:
novo_cliente_df = spark.createDataFrame(novo_cliente)

In [90]:
novo_cliente_df.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [91]:
novo_cliente_df_prep = assembler.transform(novo_cliente_df).select('features')

In [92]:
novo_cliente_df_prep.show(truncate=False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



In [94]:
# previsão para novo cliente
modelo_rfc_tuning.transform(novo_cliente_df_prep).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[4.97193078277745...|[0.06214913478471...|       1.0|
+--------------------+--------------------+--------------------+----------+

